**Esse arquivo é ignorado pelo Git. O script somente deve ser usado para testar gráficos antes de inserir no arquivo indicadores.py**

In [2]:
import pandas as pd
import requests
import sidrapy
import numpy as np
import scripts_saneamento
import datetime as dt

In [130]:
ipca = scripts_saneamento.importa_ipca('ipca_202210SerieHist.xls')

ipca['Mês'] = pd.to_datetime(ipca['Data'], format='%m/%Y')

ipca.set_index('Mês', inplace=True)

ipca['index'] = ipca.index.year

ipca['IPCA - Variação mensal'] = ipca['Índice'].pct_change(periods=1)*100

ipca['IPCA - Variação acumulada em 12 meses'] = ipca['Índice'].pct_change(periods=12)*100

ipca["IPCA - Variação acumulada no ano"] = ipca.groupby([ipca.index.year])["IPCA - Variação mensal"].transform(sum)

for mes in ipca.index.month:
    ipca['IPCA - Variação acumulada no ano'] = ipca['IPCA - Variação mensal'].rolling(mes).sum()

ipca['ano_base'] = ipca['Índice'].groupby(ipca.index.year).transform('last')

ipca['ano_base'] = ipca['ano_base'].shift(12)

ipca['IPCA - Variação acumulada no ano'] = (ipca['Índice']/ipca['ano_base'] - 1) * 100


ipca = ipca[(ipca.index.year > 1994) & (ipca.index.year < 2022)].drop(['index', 'Índice', 'ano_base', 'Data'], axis=1)

ipca['IPCA - Variação acumulada no ano'] = pd.to_numeric(ipca['IPCA - Variação acumulada no ano'])

ipca['IPCA - Variação acumulada no ano'] = ipca['IPCA - Variação acumulada no ano'].round(2)


In [131]:
ipca.reset_index(inplace=True)
ipca.to_feather('ipca_brasil_94_21.feather')

In [85]:
data_atual = dt.datetime.now() # captura data para determinar período na consulta da API
ipca_brasil = requests.get(f'https://apisidra.ibge.gov.br/values/t/7060/n1/all/v/63,69,2265/p/202201-{data_atual.year}{data_atual.month-1}/c315/7169/d/v63%202')
ipca_brasil = ipca_brasil.json()
ipca_brasil = pd.DataFrame(ipca_brasil)

## Tratamento dos dados do IPCA (Variação Mensal, Acumulado no Ano e Acumulado em 12 Meses)

ipca_brasil.columns = ipca_brasil.iloc[0] 
ipca_brasil = ipca_brasil[1:]
ipca_brasil['Valor'] = ipca_brasil['Valor'].astype(float)
ipca_brasil['Mês (Código)'] = pd.to_datetime(ipca_brasil['Mês (Código)'], format='%Y%m')

ipca_brasil = pd.crosstab(ipca_brasil['Mês (Código)'], ipca_brasil['Variável'], ipca_brasil['Valor'], aggfunc='sum')

coluna_ordem = ['IPCA - Variação mensal', 'IPCA - Variação acumulada em 12 meses', 'IPCA - Variação acumulada no ano']

ipca_brasil = ipca_brasil.reindex(coluna_ordem, axis=1)

ipca_brasil_94_21 = pd.read_feather('ipca_brasil_94_21.feather')

ipca_brasil_94_21.set_index('Mês', inplace=True)

ipca_brasil = pd.concat([ipca_brasil_94_21, ipca_brasil], axis=0)

In [144]:
ipca_brasil.index.max().year

2022

In [145]:
ipca_brasil[(ipca_brasil.index >= f'{ipca_brasil.index.max().year}-01-01')]

,IPCA - Variação mensal,IPCA - Variação acumulada em 12 meses,IPCA - Variação acumulada no ano
2022-01-01,0.54,10.38,0.54
2022-02-01,1.01,10.54,1.56
2022-03-01,1.62,11.30,3.20
2022-04-01,1.06,12.13,4.29
2022-05-01,0.47,11.73,4.78
2022-06-01,0.67,11.89,5.49
2022-07-01,-0.68,10.07,4.77
2022-08-01,-0.36,8.73,4.39
2022-09-01,-0.29,7.17,4.09
2022-10-01,0.59,6.47,4.70
